In [6]:
import pandas as pd
from arm import MSTARM

In [7]:
df = pd.read_csv('data/output/class_rules.csv')
df.head()

,a,b,support,confidence,lift
0,"('Ethanol', 'Cigarettes')","('Diesel Auto Clean',)",0.010214,0.461185,2.563858
1,"('Ethanol', 'Cigarettes')","('Special Gasoline', 'Diesel')",0.006097,0.275286,4.098598
2,"('Ethanol', 'Cigarettes')","('Diesel',)",0.006118,0.276232,3.214333
3,"('Ethanol', 'Cigarettes')","('Special Gasoline', 'Diesel Auto Clean')",0.010176,0.459476,3.834119
4,"('Ethanol', 'Cigarettes')","('Special Gasoline',)",0.021821,0.985283,1.356976


In [12]:
df.shape

(100, 5)

### By highest `support` score

In [9]:
df.sort_values(by=['support'], ascending=False).head()

,a,b,support,confidence,lift
31,"('Diesel Auto Clean',)","('Special Gasoline',)",0.119839,0.666217,0.917544
81,"('Ethanol',)","('Special Gasoline',)",0.109000,0.881073,1.213453
94,"('Special Diesel',)","('Gasoline',)",0.080917,0.625656,1.150017
91,"('Lubricant',)","('Gasoline',)",0.071294,0.771021,1.417212
67,"('Cigarettes',)","('Special Gasoline',)",0.067925,0.954123,1.314061


### By highest `confidence` score

In [11]:
df.sort_values(by=['confidence'], ascending=False).head()

,a,b,support,confidence,lift
60,"('Ethanol', 'Cigarettes', 'Diesel')","('Special Gasoline',)",0.006097,0.996576,1.372529
55,"('Diesel Auto Clean', 'Ethanol', 'Cigarettes')","('Special Gasoline',)",0.010176,0.996293,1.372140
12,"('Diesel Auto Clean', 'Cigarettes', 'Diesel')","('Special Gasoline',)",0.007479,0.996237,1.372062
14,"('Diesel Auto Clean', 'Ethanol', 'Diesel')","('Special Gasoline',)",0.009812,0.995515,1.371068
53,"('Cold Alcoholic Beverages', 'Diesel')","('Special Gasoline',)",0.005169,0.987081,1.359452


### By highest `lift` score

In [13]:
df.sort_values(by=['lift'], ascending=False).head()

,a,b,support,confidence,lift
87,"('Lubricant',)","('Gasoline', 'Filters')",0.009510,0.102851,10.235194
93,"('Filters',)","('Lubricant', 'Gasoline')",0.009510,0.727197,10.200036
89,"('Lubricant',)","('Filters',)",0.012233,0.132294,10.115787
98,"('Chips',)","('Juices And Soft Drinks',)",0.007060,0.330025,8.585170
99,"('Juices And Soft Drinks',)","('The Bakery',)",0.006370,0.165697,8.202814
